# 함수를 객체처럼

In [5]:
def factorial(n):
    '''returns n'''
    return 1 if n < 2 else n * factorial(n-1)

factorial(42), factorial.__doc__, type(factorial)

(1405006117752879898543142606244511569936384000000000, 'returns n', function)

In [7]:
fact = factorial
print(fact(5))

print(map(factorial, range(11)))
print(list(map(fact, range(11))))

120
[1, 1, 2, 6, 24, 120, 720, 5040, 40320, 362880, 3628800]


## 고위함수

In [9]:
fruits = ['strawberry', 'cherry', 'fig', 'apple', 'orange']
# key 인수로 함수를 사용
sorted(fruits, key=len)

['fig', 'apple', 'cherry', 'orange', 'strawberry']

In [11]:
def reverse(word):
    return word[::-1]

sorted(fruits, key=reverse)

['orange', 'apple', 'fig', 'strawberry', 'cherry']

In [12]:
list(map(fact, range(6)))
# listcomp가 더 가독성이 좋다.
[fact(n) for n in range(6)]

[1, 1, 2, 6, 24, 120]

In [13]:
list(map(factorial, filter(lambda n: n%2, range(6))))
# 아래가 더 가독성이 좋다.
[factorial(n) for n in range(6) if n % 2]


[1, 6, 120]

In [16]:
from functools import reduce
from operator import add

reduce(add, range(100))
# 가독성과 성능 모두 sum이 더 낫다.
sum(range(100))

4950

# 익명 함수

In [18]:
sorted(fruits, key=lambda x: x[::-1])

['orange', 'apple', 'fig', 'strawberry', 'cherry']

# 콜러블 객체

In [21]:
abs, str, 13
[callable(obj) for obj in (abs, str, 13)]

[True, True, False]

# 사용자 정의형 콜러블형

In [22]:
import random

class BingoCage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError("pick from empty BingoCage")
    # 콜러블
    def __call__(self):
        return self.pick()

In [32]:
bingo = BingoCage(range(3))
bingo(), callable(bingo)

(2, True)

# 위치 매개변수와 키워드 전용 매개변수

In [19]:
def tag(name, *content, cls=None, **attrs):
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attr_str = ''.join(f'{attr}="{value}"' for attr, value in sorted(attrs.items()))
    else:
        attr_str = ''
    if content:
        return '\n'.join(f'<{name}{attr_str}>{c}</{name}>' for c in content)
    else:
        return f'<{name}{attr_str} />'

In [22]:
print(tag('br'))
print(tag('p', 'hello'))
print(tag('p', 'hello', 'world'))
print(tag('p', 'hello', id=33))
print(tag('p', 'hello', 'world', cls='sidebar'))
print(tag(content='testing', name='img'))
my_tag = {'name': 'img', 'title': 'Sunset Boulevard', 'src': 'sunset.jpg', 'cls': 'framed'}
tag(**my_tag)

<br />
<p>hello</p>
<p>hello</p>
<p>world</p>
<pid="33">hello</p>
<pclass="sidebar">hello</p>
<pclass="sidebar">world</p>
<imgcontent="testing" />


'<imgclass="framed"src="sunset.jpg"title="Sunset Boulevard" />'

In [44]:
def f(a, *, b):
    return a, b

f(1, b=3)

(1, 3)

# 매개변수에 대한 정보 읽기

In [6]:
import bobo

@bobo.query('/')
def hello(person):
    return f'Hello {person}!'

In [10]:
# 인수가 빠졌으므로 에러 발생
!curl -i http://localhost:8080/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
curl: (7) Failed to connect to localhost port 8080 after 2229 ms: Connection refused


In [5]:
!curl -i http://localhost:8080/?person=Hans

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
curl: (7) Failed to connect to localhost port 8080 after 2241 ms: Connection refused


In [11]:
def clip(text, max_len=80):
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after

    if end is None:
        end = len(text)
    return text[:end].rstrip()


In [15]:
clip.__defaults__, clip.__code__.co_varnames

((80,), ('text', 'max_len', 'end', 'space_before', 'space_after'))

In [16]:
from inspect import signature

sig = signature(clip)
sig, str(sig)


(<Signature (text, max_len=80)>, '(text, max_len=80)')

In [17]:
for name, param in sig.parameters.items():
    print(param.kind, ':', name, '=', param.default)
    

POSITIONAL_OR_KEYWORD : text = <class 'inspect._empty'>
POSITIONAL_OR_KEYWORD : max_len = 80


In [24]:
sig = signature(tag)
bound_args = sig.bind(**my_tag)

for name, value in bound_args.arguments.items():
    print(name, '=', value)

name = img
cls = framed
attrs = {'title': 'Sunset Boulevard', 'src': 'sunset.jpg'}


In [25]:
del my_tag['name']
# name 변수가 없으므로 에러
bound_args = sig.bind(**my_tag)

TypeError: missing a required argument: 'name'

# 함수 애너테이션

In [26]:
def clip(text:str, max_len:'int>0'=80) -> str:
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after

    if end is None:
        end = len(text)
    return text[:end].rstrip()


In [27]:
clip.__annotations__ 

{'text': str, 'max_len': 'int>0', 'return': str}

# 함수형 프로그래밍을 위한 패키지

In [28]:
from functools import reduce
from operator import mul

def fact(n):
    return reduce(mul, range(1,n+1))

In [29]:
metro_data = [
    ('Tokyo', 'JP', 39.24, (35.3615, 139.3252)),
    ('dqwf', 'EWR', 3.32, (35.3615, 139.3252)),
    ('zdfq', 'XC', 52.57, (35.3615, 139.3252)),
    ('tq43t', 'FE', 75.21, (35.3615, 139.3252)),
    ('zcxv', 'QY', 83.14, (35.3615, 139.3252))

]
from operator import itemgetter
for city in sorted(metro_data, key=itemgetter(1)):
    print(city)

<>:2: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
<>:2: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
C:\Users\hans\AppData\Local\Temp/ipykernel_3260/3860820359.py:2: SyntaxWarning: 'tuple' object is not callable; perhaps you missed a comma?
  ('Tokyo', 'JP', 39.24, (35.3615, 139.3252))


TypeError: 'tuple' object is not callable